# **Group 14: Extended Wolf, Goat, Cabbage Problem** 


# BASE

In [ ]:
import random
from math import inf
from statistics import mean

objects = ['Shepherd', 'Wolf', 'Goat', 'Cabbage', 'Stick', 'Torch']
conflicts = [('Wolf', 'Goat'), ('Goat', 'Cabbage'), ('Wolf', 'Stick'), ('Stick', 'Torch')]

initial_state = ('Cabbage', 'Goat', 'Shepherd', 'Stick', 'Torch', 'Wolf')
goal = ()

shuffle_states = False
next_states = {}

def find_next_states(x : tuple):
    if shuffle_states:
        return next_states[x]
    x = list(x)
    states = []
    if 'Shepherd' in x:
        x.remove('Shepherd')
        states.append(tuple(x))
        for i in range(len(x)):
            tmp = x[:]; tmp.pop(i)
            states.append(tuple(tmp))
            for j in range(i, len(tmp)):
                tmp2 = tmp[:]; tmp2.pop(j)
                states.append(tuple(tmp2))
    else:
        x.append('Shepherd'); x.sort()
        states.append(tuple(x))
        for i in range(len(objects)):
            if objects[i] not in x:
                tmp = x[:]; tmp.append(objects[i]); tmp.sort()
                states.append(tuple(tmp))
                for j in range(i + 1, len(objects)):
                    if objects[j] not in x:
                        tmp2 = tmp[:]; tmp2.append(objects[j]); tmp2.sort()
                        states.append(tuple(tmp2))
    return states

def valid_state(x : tuple):
    for conflict in conflicts:
        if (conflict[0] in x and conflict[1] in x and 'Shepherd' not in x) or (conflict[0] not in x and conflict[1] not in x and 'Shepherd' in x):
            return False
    return True

def random_all_next_states(state):
    l = find_next_states(state)
    random.shuffle(l)
    next_states[state] = l
    for next_state in l:
        if next_state not in next_states:
            random_all_next_states(next_state)

class ProblemSolver:
    def __init__(self):
        self.name = ''
        self.maxTime = self.maxSpace = 0
        self.minTime = self.minSpace = inf
        self.time = []
        self.space = []
        self.result = []
    
    def run(self):
        pass

    def solve(self):
        self.run()
        self.maxTime = max(self.maxTime, self.time[-1])
        self.maxSpace = max(self.maxSpace, self.space[-1])
        self.minTime = min(self.minTime, self.time[-1])
        self.minSpace = min(self.minSpace, self.space[-1])
    
    def get_max_time(self):
        return self.maxTime
    
    def get_min_time(self):
        return self.minTime
    
    def get_max_space(self):
        return self.maxSpace

    def get_min_space(self):
        return self.minSpace
    
    def get_avg_time(self):
        return mean(self.time)
    
    def get_avg_space(self):
        return mean(self.space)

# BREADTH FIRST SEARCH

In [ ]:

import collections

class BfsTree(ProblemSolver):
    def __init__(self):
        super().__init__()
        self.name = 'BFS tree'

    def run(self):
        time = space = 0
        result = None
        queue = collections.deque()
        queue.append((initial_state, 0, None))
        while len(queue) > 0:
            time += 1
            space = max(space, len(queue))
            current = queue.popleft()
            if current[0] == goal:
                result = current[1]
                break
            for state in find_next_states(current[0]):
                if not valid_state(state) or state == current[2]:
                    continue
                queue.append((state, current[1] + 1, current[0]))
        self.time.append(time)
        self.space.append(space)
        self.result.append(result)

class Bfs(ProblemSolver):
    def __init__(self):
        super().__init__()
        self.name = 'BFS graph'

    def run(self):
        time = space = 0
        visited = {initial_state}
        result = None
        queue = collections.deque()
        queue.append((initial_state, 0))
        while len(queue) > 0:
            time += 1
            space = max(space, len(queue) + len(visited))
            current = queue.popleft()
            if current[0] == goal:
                result = current[1]
                break
            for state in find_next_states(current[0]):
                if (not valid_state(state)) or (state in visited):
                    continue
                visited.add(state)
                queue.append((state, current[1] + 1))
        self.time.append(time)
        self.space.append(space)
        self.result.append(result)

# ITERATIVE DEEPENING SEARCH

In [ ]:

class Ids(ProblemSolver):
    def __init__(self):
        super().__init__()
        self.name = 'IDS'

    def dls(self, limit):
        self.ctime = 0
        self.cspace = 0
        def limited_depth(current, parent, cost, limit, space):
            self.ctime += 1
            self.cspace = max(self.cspace, space)
            if cost > limit:
                return None
            if current == goal:
                return cost
            state_list = find_next_states(current)
            for next in find_next_states(current):
                if next != parent and valid_state(next):
                    result = limited_depth(next, current, cost + 1, limit, space + len(state_list) + 1)
                    if result != None:
                        return result
        return limited_depth(initial_state, None, 0, limit, 1)

    def run(self):
        time = space = 0
        result = 0
        while True:
            stop = self.dls(result)
            time += self.ctime
            space = max(space, self.cspace)
            if stop != None:
                break
            result += 1
        self.time.append(time)
        self.space.append(space)
        self.result.append(result)

# A*SEARCH

In [ ]:
import heapq
''
class PriorityQueue:
    def __init__(self):
        self.state = []

    def push(self, state, cost):
        heapq.heappush(self.state, (cost, state))
        
    def pop(self):
        return heapq.heappop(self.state)[1]
    def empty(self):
        return len(self.state) < 1
    
    def size(self):
        return len(self.state)

class Astar(ProblemSolver):
    def __init__(self):
        super().__init__()
        self.name = 'A*'

    def heuristic(self, current_state):
        if 'Shepherd' in current_state:
            h_value = (len(current_state) // 2) * 2 - 1
        else:
            h_value = ((len(current_state) + 1) // 2) * 2
        return int(h_value)

    def run(self):
        time = space = 0
        q = PriorityQueue()
        q.push(initial_state, self.heuristic(initial_state))
        depth = {initial_state: 0}
        while not q.empty():
            time += 1
            space = max(space, q.size() + len(depth))
            current = q.pop()
            if current == goal:
                break
            for next in find_next_states(current):
                if next not in depth and valid_state(next):
                    depth[next] = depth[current] + 1
                    f_cost = depth[next] + self.heuristic(next)
                    q.push(next, f_cost)
        self.time.append(time)
        self.space.append(space)
        self.result.append(depth[goal])

class AstarTree(ProblemSolver):
    def __init__(self):
        super().__init__()
        self.name = 'A* tree'
        self.queue = []

    def heuristic(self, current_state):
        if 'Shepherd' in current_state:
            h_value = (len(current_state) // 2) * 2 - 1
        else:
            h_value = ((len(current_state) + 1) // 2) * 2
        return int(h_value)
    def run(self):
        time = space = 0
        result = None
        self.queue.clear()
        heapq.heappush(self.queue, (self.heuristic(initial_state), 0, initial_state, None))
        while len(self.queue) > 0:
            time += 1
            space = max(space, len(self.queue))
            current = heapq.heappop(self.queue)
            if current[2] == goal:
                result = current[1]
                break
            for next in find_next_states(current[2]):
                if next != current[3] and valid_state(next):
                    f_cost = current[1] + 1 + self.heuristic(next)
                    heapq.heappush(self.queue, (f_cost, current[1] + 1, next, current[2]))
        self.time.append(time)
        self.space.append(space)
        self.result.append(result)

# IDA* SEARCH

In [ ]:


class Idastar(ProblemSolver):
    def __init__(self):
        super().__init__()
        self.name = 'IDA*'

    def heuristic(self, current_state):
        if 'Sheperd' in current_state:
            h_value = (len(current_state) // 2) * 2 - 1
        else:
            h_value = ((len(current_state) + 1) // 2) * 2
        return int(h_value)

    def dfs(self, bound):
        self.ctime = 0
        self.cspace = 0
        self.cresult = None
        def search(node, parent, g, bound, space):
            self.ctime += 1
            self.cspace = max(self.cspace, space)
            f = g + self.heuristic(node)
            if f > bound:
                return f
            if node == goal:
                self.cresult = g
                return
            new_bound = float('inf')
            state_list = find_next_states(node)
            state_list.sort(key=lambda x : self.heuristic(x))
            for next_node in state_list:
                if next_node != parent and valid_state(next_node):
                    t = search(next_node, node, g + 1, bound, space + len(state_list) + 1)
                    if self.cresult != None:
                        return
                    new_bound = min(new_bound, t)
            return new_bound
        return search(initial_state, None, 0, bound, 1)

    def run(self):
        time = space = 0
        bound = self.heuristic(initial_state)
        while True:
            t = self.dfs(bound)
            time += self.ctime
            space = max(space, self.cspace)
            if self.cresult != None:
                break
            bound = t
        self.time.append(time)
        self.space.append(space)
        self.result.append(self.cresult)


# MAIN


In [ ]:
import random
import sys
import base, bfs, ids, astar, idastar

def read_file():
    states = []
    with open('all.txt', 'r') as f:
        for line in f:
            states.append(eval(line.strip()))
    return states

def analysis(test_count, algorithms: list, trace = False):
    table = [['Algorithm', 'Max time', 'Min time', 'Avg time', 'Max space', 'Min space', 'Avg space']]
    for algo in algorithms:
        column = [algo.name, \
            algo.get_max_time(), algo.get_min_time(), algo.get_avg_time(), \
            algo.get_max_space(), algo.get_min_space(), algo.get_avg_space()]
        table.append(column)
    rows = ['|'] * 7
    for col in table:
        maxLen = 0
        for value in col:
            maxLen = max(maxLen, len(str(value)))
        for i in range(7):
            label = str(col[i])
            rows[i] += ' ' + label + ' ' * (maxLen - len(label)) + ' |'
    print('Number of tests:', test_count)
    for row in rows:
        print(row)
    if trace:
        print()
        for algo in algorithms:
            print('* Algorithm: ' + algo.name + ' - Number of steps:', algo.result[0])
            print()
            j = 0
            for state in algo.trace_back():
                print(str(j) + '. ', end='')
                sideA, sideB = '', ''
                for i in base.objects:
                    if i in state:
                        sideA += i + ' '
                    else:
                        sideB += i + ' '
                print('-' * 42)
                print('   ' + '| {:38} |'.format(sideA.strip()))
                print('   ' + '-' * 42)
                print('   ' + '| {:38} |'.format(sideB.strip()))
                print('   ' + '-' * 42)
                j += 1
            print()

if __name__ == "__main__":
    notebook_argv = "-algo idastar -trace-back"
    argv = notebook_argv.split()
    print(argv)
    # argv = sys.argv
    test_count = 50
    all_algo = {'bfstree': bfs.BfsTree(), 'bfs': bfs.Bfs(), 'ids': ids.Ids(), \
                'astartree': astar.AstarTree(), 'astar': astar.Astar(), 'idastar': idastar.Idastar()}
    algorithms = []
    initial_states = read_file()
    trace_back = False
    i = 0
    while i < len(argv):
        if argv[i] == '-test-count':
            test_count = int(argv[i + 1])
            i += 2
        elif argv[i] == '-algo':
            for algo in argv[i + 1].split(','):
                algorithms.append(all_algo[algo])
            i += 2
        elif argv[i] == '-trace-back':
            trace_back = True
            initial_states = [('Cabbage', 'Goat', 'Shepherd', 'Stick', 'Torch', 'Wolf')]
            i += 1
        else:
            print('Invalid argument')
            exit(1)
    if len(algorithms) < 1:
        algorithms = [value for value in all_algo.values()]
    if trace_back:
        test_count = 1
    for test in range(test_count):
        base.next_states.clear()
        base.shuffle_states = False
        base.random_all_next_states(())
        base.shuffle_states = True
        base.initial_state = initial_states[random.randrange(0, len(initial_states))]
        for algo in algorithms:
            algo.solve()
    analysis(test_count, algorithms, trace_back)

['-algo', 'idastar', '-trace-back']
Number of tests: 1
| Algorithm | IDA* |
| Max time  | 1080 |
| Min time  | 1080 |
| Avg time  | 1080 |
| Max space | 79   |
| Min space | 79   |
| Avg space | 79   |

* Algorithm: IDA* - Number of steps: 7

0. ------------------------------------------
   | Shepherd Wolf Goat Cabbage Stick Torch |
   ------------------------------------------
   |                                        |
   ------------------------------------------
1. ------------------------------------------
   | Wolf Cabbage Torch                     |
   ------------------------------------------
   | Shepherd Goat Stick                    |
   ------------------------------------------
2. ------------------------------------------
   | Shepherd Wolf Cabbage Torch            |
   ------------------------------------------
   | Goat Stick                             |
   ------------------------------------------
3. ------------------------------------------
   | Wolf            

#RUNNING 50 TESTS

In [ ]:
test_count = 50
algorithms = [BfsTree(), Bfs(), Ids(), AstarTree(), Astar(), Idastar()]
initial_states = read_file()

for test in range(test_count):
    next_states.clear()
    shuffle_states = False
    random_all_next_states(())
    shuffle_states = True
    initial_state = initial_states[random.randrange(0, len(initial_states))]
    for algo in algorithms:
        algo.solve()
analysis(test_count, algorithms)

Number of tests: 50
| Algorithm | BFS tree | BFS   | IDS    | A* tree | A*    | IDA*   |
| Max time  | 1009     | 26    | 5075   | 272     | 20    | 1229   |
| Min time  | 1        | 1     | 1      | 1       | 1     | 1      |
| Avg time  | 189.5    | 21.04 | 948.36 | 47.78   | 9.28  | 257.36 |
| Max space | 2675     | 39    | 83     | 712     | 38    | 83     |
| Min space | 1        | 2     | 1      | 1       | 2     | 1      |
| Avg space | 500.22   | 32    | 44.84  | 124.62  | 28.22 | 43.44  |
